# 网络书籍下载notebook

## define book.json structure
``` json
{
  "title": "诗经",
  "description": "",
  "source": "",
  "volumes": [
    {
      "title": "国风·周南",
      "source": "",
      "chapters": [
        {
          "title": "关雎",
          "source": "",
          "paragraphs": [
            {
              "type": "原文",
              "content": "关关雎鸠，在河之洲。窈窕淑女，君子好逑。"
            },
            {
              "type": "原文",
              "content": "参差荇菜，左右流之。窈窕淑女，寤寐求之。"
            },
            {
              "type": "原文",
              "content": "求之不得，寤寐思服。悠哉悠哉，辗转反侧。"
            },
            {
              "type": "原文",
              "content": "参差荇菜，左右采之。窈窕淑女，琴瑟友之。"
            },
            {
              "type": "原文",
              "content": "参差荇菜，左右芼之。窈窕淑女，钟鼓乐之。"
            }
          ]
        },
        {
          "title": "葛覃",
          "source": "",
          "paragraphs": [
            {
              "type": "原文",
              "content": "葛之覃兮，施于中谷，维叶萋萋。黄鸟于飞，集于灌木，其鸣喈喈。"
            },
            {
              "type": "原文",
              "content": "葛之覃兮，施于中谷，维叶莫莫。是刈是濩，为絺为綌，服之无斁。"
            },
            {
              "type": "原文",
              "content": "言告师氏，言告言归。薄污我私。薄浣我衣。害浣害否？归宁父母。"
            }
          ]
        }
      ]
    },
    {
      "title": "国风·召南",
      "chapters": [
        {
          "title": "鹊巢",
          "paragraphs": [
            {
              "type": "原文",
              "content": "维鹊有巢，维鸠居之。之子于归，百两御之。"
            },
            {
              "type": "原文",
              "content": "维鹊有巢，维鸠方之。之子于归，百两将之。"
            },
            {
              "type": "原文",
              "content": "维鹊有巢，维鸠盈之。之子于归，百两成之。"
            }
          ]
        },
        {
          "title": "甘棠",
          "paragraphs": [
            {
              "type": "原文",
              "content": "蔽芾甘棠，勿翦勿伐，召伯所茇。"
            },
            {
              "type": "原文",
              "content": "蔽芾甘棠，勿翦勿败，召伯所憩。"
            },
            {
              "type": "原文",
              "content": "蔽芾甘棠，勿翦勿拜，召伯所说。"
            }
          ]
        }
      ]
    }
  ]
}
```

In [68]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 703.1 kB/s eta 0:00:0000:0100:01


In [84]:
import os
import chardet
import requests
from bs4 import BeautifulSoup

In [85]:
outputfile="/Users/sunyafu/zebra/YIJING/Books/"

In [86]:
def geturlcontent(url):
    
    # 发送HTTP GET请求获取网页内容
    print(f"download: {url}")
    response = requests.get(url)

    if response.status_code == 200:
        #print(f"text: {response.text}")
        #print(f"status code: {response.status_code}")
        
        #print(f"encoding: {response.encoding}")
        #print(f"apparent encoding: {response.apparent_encoding}")
        chardet_encoding = chardet.detect(response.content);
        #print(f"chardet encoding: {chardet_encoding}"
        
        # 调整编码，避免requests编码识别问题。
        # encoding对中文的编码识别，会识别为：ISO-8859-1。
        # apparent_encoding也会识别错误，会出现：mac_greek。
        # chardet能识别为GB2312，还是会判断错误，会把GB18030判断为GB2312，后果就是，部分中文有错误。
        #response.encoding = response.apparent_encoding
        #response.encoding = chardet_encoding['encoding']
        #response.encoding = 'GB18030'
        if response.apparent_encoding == "GB18030" or chardet_encoding['encoding'] == "GB2312":
            response.encoding = 'GB18030'
        else:
            response.encoding = chardet_encoding['encoding']
        print(f"encoding: {response.encoding}")
        #print(f"text: {response.text}")
        
        return response;
    else:
        print(f"download error: status_code {response.status_code}.")
        
    return None

In [87]:
def decodecontentaschapter(url, class_):

    # Decode为JSON文件
    response = geturlcontent(url)
    if response is not None:
        chapter = dict()
        
        # 解析网页内容
        soup = BeautifulSoup(response.text, "html.parser")

        # 提取目标内容
        h1 = soup.find("h1")
        title = h1.get_text(strip = True)
        if title is not None:
            chapter["title"] = title
        else:
            chapter["title"] = ""
        print(title)
        chapter["source"] = url
        
        div = soup.find("div", class_ = class_)
        ps = div.find_all("p");
        if len(ps) > 0:
            paragraphs = []
            for p in ps:
                c = p.get_text(strip = False)
                if len(c):
                    paragraph = {}
                    paragraph["type"] = "原文"
                    paragraph["content"] = c
                    paragraphs.append(paragraph)
            if len(paragraphs):
                chapter["paragraphs"] = paragraphs
        else:
            paragraphs = []
            c = div.get_text(strip = False)
            if len(c):
                paragraph = {}
                paragraph["type"] = "原文"
                paragraph["content"] = c
                paragraphs.append(paragraph)
            if len(paragraphs):
                chapter["paragraphs"] = paragraphs
        
        return chapter;
    
    return None

In [88]:
def getbookurls(baseurl, bookid, startpage, endpage):
    
    urls = []
    for id in range(startpage, endpage + 1):
        urls.append(f"{baseurl}{bookid}/{id}.html")
    return urls

In [89]:
def downloadbook(outputfile, baseurl, bookid, startpage, endpage, class_="width"):
    
    urls = getbookurls(baseurl, bookid, startpage, endpage)
    
    # 循环获取不同ID的内容并保存
    with open(outputfile, "w", encoding="utf-8") as file:
        for url in urls:
            content = geturlcontent(url, class_);
            if content is not None:
                file.write(content + "\n");
                
    print(f"内容已保存到{outputfile}文件中。")

In [103]:
def getbookurlsfromgushiwen(url):

    # Decode为JSON文件
    response = geturlcontent(url)
    if response is not None:
        book = dict()
        
        # 解析网页内容
        soup = BeautifulSoup(response.text, "html.parser")

        # decode title
        h1 = soup.find("h1")
        title = h1.get_text(strip = True)
        if title is not None:
            book["title"] = title
        else:
            book["title"] = ""
        print(title)
        book["source"] = url

        # decode description
        div = soup.find("div", class_="sonspic", id = "sonsyuanwen")
        div = div.find("div", class_="cont")
        p = div.find_all("p");
        description = p[0].get_text(strip = True)
        if description is not None:
            book["description"] = description
        else:
            book["description"] = ""
        
        # decode volumes    
        volumes = []
        div = soup.find("div", class_="sons")
        div_volumes = div.find_all("div", class_="bookcont");
        
        for div_volume in div_volumes:
            volume = {}
            div_volume_title = div_volume.find(class_="bookMl")
            title = None
            if div_volume_title is not None:
                title = div_volume_title.get_text(strip = True)
            if title is not None:
                volume["title"] = title
            else:
                volume["title"] = ""

            chapters = []
            linkss = div_volume.find_all("a");
            for link in linkss:
                chapter = {}
                chapter["title"] = link.get_text(strip = True)
                chapter["source"] = link.attrs["href"]
                chapters.append(chapter)
            volume["chapters"] = chapters
            volumes.append(volume)
        book["volumes"] = volumes
            
        return book
    
    return None

In [104]:
book = getbookurlsfromgushiwen("https://so.gushiwen.cn/guwen/book_46653FD803893E4F75696240258265D2.aspx")

download: https://so.gushiwen.cn/guwen/book_46653FD803893E4F75696240258265D2.aspx
encoding: utf-8
论语


In [20]:
def downloadbook2(outputfile, urls, class_="contson"):
    
    # 循环获取不同ID的内容并保存
    with open(outputfile, "w", encoding="utf-8") as file:
        for url in urls:
            content = geturlcontent(url, class_);
            if content is not None:
                file.write(content + "\n");
                
    print(f"内容已保存到{outputfile}文件中。")

In [21]:
#downloadbook2(outputfile + "礼记.txt", ["https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F6FAFA1EF2868E474.aspx"])

In [22]:
#下载《诗经》
#downloadbook(outputfile+"诗经.txt", "http://www.mingzhuxiaoshuo.com/xianqin/", 17, 438, 466)

In [23]:
#下载《尚书》
#downloadbook(outputfile + "尚书.txt", "http://www.mingzhuxiaoshuo.com/xianqin/", 24, 598, 654)

In [24]:
#下载《逸周书》
#downloadbook(outputfile + "逸周书.txt", "https://www.tadu.com/book/", 401311, 24391454, 24391467)

In [25]:
# 下载《礼记》
urls = getbookurls2("https://so.gushiwen.cn/guwen/book_46653FD803893E4F4229823F35D89DC5.aspx")
downloadbook2(outputfile + "礼记.txt", urls)

encoding: utf-8
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F90087631A264778C.aspx
encoding: utf-8
曲礼上原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F992004DD8137A757.aspx
encoding: utf-8
曲礼下原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F96A9628A5D51AF7E.aspx
encoding: utf-8
檀弓上原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F5676BF144E5EC5FB.aspx
encoding: utf-8
檀弓下原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4FCE5E622043A933FE.aspx
encoding: utf-8
王制原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F070BF445D39262F8.aspx
encoding: utf-8
月令原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F6FAFA1EF2868E474.aspx
encoding: utf-8
曾子问原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4FEE5973D3D49B201F.aspx
encoding: utf-8
文王世子原文原文
download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4FAFFF8D914EAA516C.aspx
encoding: utf-8
礼运原文原文
download: https://so.gu

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
books_from_gushiwen = [
    {
        "name": "诗经",
        "directory": "https://so.gushiwen.cn/gushi/shijing.aspx"
    },
    {
        "name": "尔雅",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F1ED72C2149EB00A0.aspx"
    },
    {
        "name": "周礼",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F89215E11979D4F9F.aspx"
    },
    {
        "name": "仪礼",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F5BD7D6F638166051.aspx"
    },
    {
        "name": "礼记",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F4229823F35D89DC5.aspx"
    },
    {
        "name": "大戴礼记",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F59554702CBAB8327.aspx"
    },
    {
        "name": "左传",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F699E8628DEAEE3C0.aspx"
    },
    {
        "name": "公羊传",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F1C34B0BCE27385A0.aspx"
    },
    {
        "name": "谷梁传",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F1AEF8EC5EB7CD7B8.aspx"
    },
    {
        "name": "国语",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F8019EA9A29165384.aspx"
    },
    {
        "name": "战国策",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F90B30EC6FDD22553.aspx"
    },
    {
        "name": "竹书纪年",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F03DA4E2FE601D843.aspx"
    },
    {
        "name": "穆天子传",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F4E9CADD37301AA08.aspx"
    },
    {
        "name": "山海经",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F6FCB61241D8108D0.aspx"
    },
    {
        "name": "易传",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F68FD6588DE2463D0.aspx"
    },
    {
        "name": "黄帝内经",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4FF001D427402FA6E0.aspx"
    },
    {
        "name": "难经",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4FAA70382D08D37221.aspx"
    },
    {
        "name": "老子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F33D126D4A6B656E2.aspx"
    },
    {
        "name": "论语",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F75696240258265D2.aspx"
    },
    {
        "name": "孟子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F594CDFAF65570422.aspx"
    },
    {
        "name": "荀子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F2B41025A041B0688.aspx"
    },
    {
        "name": "孝经",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F640E24BD63B0F779.aspx"
    },
    {
        "name": "墨子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F8BCB0149097D7698.aspx"
    },
    {
        "name": "庄子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4F92BED6BFD51EA958.aspx"
    },
    {
        "name": "文子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4FE6B070719E9F54EA.aspx"
    },
    {
        "name": "列子",
        "directory": "https://so.gushiwen.cn/guwen/book_46653FD803893E4FEC967FA5CC60AB13.aspx"
    },
    {
        "name": "",
        "directory": ""
    },
    {
        "name": "",
        "directory": ""
    },
    
]
        
             

In [40]:
chapters = geturlcontent2("https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F699E8628DEAEE3C0.aspx", class_="contson")
chapters

download: https://so.gushiwen.cn/guwen/bookv_46653FD803893E4F699E8628DEAEE3C0.aspx
encoding: utf-8
学而篇原文原文


{'title': '学而篇原文原文',
 'paragraphs': [{'type': '原文',
   'content': '\u3000\u3000子曰：“学而时习之，不亦说乎？有朋自远方来，不亦乐乎？人不知而不愠，不亦君子乎？”'},
  {'type': '原文',
   'content': '\u3000\u3000有子曰：“其为人也孝弟，而好犯上者，鲜矣；不好犯上而好作乱者，未之有也。君子务本，本立而道生。孝弟也者，其为仁之本与！”'},
  {'type': '原文', 'content': '\u3000\u3000子曰：“巧言令色，鲜矣仁！”'},
  {'type': '原文', 'content': '\u3000\u3000曾子曰：“吾日三省吾身：为人谋而不忠乎？与朋友交而不信乎？传不习乎？”'},
  {'type': '原文', 'content': '\u3000\u3000子曰：“道千乘之国，敬事而信，节用而爱人，使民以时。”'},
  {'type': '原文',
   'content': '\u3000\u3000子曰：“弟子入则孝，出则弟，谨而信，泛爱众，而亲仁，行有余力，则以学文。”'},
  {'type': '原文',
   'content': '\u3000\u3000子夏曰：“贤贤易色；事父母，能竭其力；事君，能致其身；与朋友交，言而有信。虽曰未学，吾必谓之学矣。”'},
  {'type': '原文',
   'content': '\u3000\u3000子曰：“君子不重则不威，学则不固。主忠信，无友不如己者，过，则勿惮改。”'},
  {'type': '原文', 'content': '\u3000\u3000曾子曰：“慎终追远，民德归厚矣。”'},
  {'type': '原文',
   'content': '\u3000\u3000子禽问于子贡曰：“夫子至于是邦也，必闻其政，求之与，抑与之与？”子贡曰：“夫子温、良、恭、俭、让以得之。夫子之求之也，其诸异乎人之求之与？”'},
  {'type': '原文', 'content': '\u3000\u3000子曰：“父在，观其志；父没，观其行；三年无改于父之道，可谓孝矣。”'},
  {'type': '原文',
   'content': '